In [ ]:
import sys
sys.path.append("/home/ailie/Repos/BBAttacks/attacks/")
sys.path.append("/home/ailie/Repos/BBAttacks/utils/")

import utils
from data_manager import load_data

# Black Box Attacks
import random_noise
import EvoStrategy
import SimbaWrapper

In [ ]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import backend as K
from tensorflow import keras as keras
importlib.reload(EvoStrategy)
import json
import time
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import sys
import gc

In [ ]:
import tensorflow as tf
DEVICE = '/GPU:1'

In [ ]:
CIFAR100_VGG_PATH = "/home/ailie/Repos/BBAttacks/models/cifar100/vgg/"
sys.path.append(CIFAR100_VGG_PATH)
import cifar100vgg

In [ ]:
importlib.reload(cifar100vgg)
WEIGHTS_FILE_NAME = "cifar100vgg.h5"
model = cifar100vgg.cifar100vgg(train=False, weights_path=CIFAR100_VGG_PATH + WEIGHTS_FILE_NAME)

In [ ]:
NUM_CLASSES = 100

In [ ]:
LOAD_DATA = True
SAVE_DATA = False

In [ ]:
if LOAD_DATA:
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)
    
    if SAVE_DATA:
        np.save("x_test.npy", x_test)
        np.save("y_test.npy", y_test)

In [ ]:
x_train = x_train.astype('int')
x_test = x_test.astype('int')

In [ ]:
gc.collect()

<h4> Check the training accuracy </h4>

In [ ]:
with tf.device(DEVICE):
    preds_train = model.predict(x_train)
    
preds_train_labels = np.argmax(preds_train, axis=1)

In [ ]:
true_train_labels = np.argmax(y_train, axis=1)

In [ ]:
acc_train = np.mean(true_train_labels == preds_train_labels)
print(f"Training accuracy: {acc_train}")

<h4> Check the test accuracy </h4>

In [ ]:
with tf.device(DEVICE):
    preds_test = model.predict(x_test)

preds_test_labels = np.argmax(preds_test, axis=1)

In [ ]:
true_test_labels = np.argmax(y_test, axis=1)

In [ ]:
acc_test = np.mean(true_test_labels == preds_test_labels)
print(f"Test accuracy: {acc_test}")

<h4> Save the images classified correctly by the network </h4>

In [ ]:
x_test_correct = x_test[preds_test_labels == true_test_labels]
y_test_correct = y_test[preds_test_labels == true_test_labels]

<h3> Run baseline EvoBA </h3>

In [ ]:
importlib.reload(EvoStrategy)

In [ ]:
GENERATION_SIZE = 30
PIXEL_COUNT = 1
STEPS = 80

In [ ]:
LEFT_IDX_ATTACK_IMAGES = 0
RIGHT_IDX_ATTACK_IMAGES = 20
SAMPLE_IMAGES = x_test_correct[LEFT_IDX_ATTACK_IMAGES: RIGHT_IDX_ATTACK_IMAGES]
SAMPLE_Y = y_test_correct[LEFT_IDX_ATTACK_IMAGES: RIGHT_IDX_ATTACK_IMAGES]

In [ ]:
VERBOSE = False

In [ ]:
%%time

with tf.device(DEVICE):
    adv_evo_strategy = {}
    
    for index in tqdm(range(len(SAMPLE_IMAGES))):
        img = SAMPLE_IMAGES[index]
        label = SAMPLE_Y[index]
        
        true_label = np.argmax(label)

        adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
            model=model,
            img=img,
            label=true_label,
            generation_size=GENERATION_SIZE, 
            one_step_perturbation_pixel_count=PIXEL_COUNT,
            verbose=VERBOSE,
            zero_one_scale=False
        )

        no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=STEPS) 

In [ ]:
def get_evoba_stats(adv_evo_strategy):
    count_succ = 0
    queries_succ = []
    l0_dists_succ = []
    indices_succ = []

    count_fail = 0
    indices_fails = []

    for i in tqdm(range(len(adv_evo_strategy))):
        img = adv_evo_strategy[i].img

        if adv_evo_strategy[i].stop_criterion():
            count_succ += 1
            queries_succ.append(adv_evo_strategy[i].queries)
            l0_dists_succ.append(np.sum(adv_evo_strategy[i].get_best_candidate() != img))
            index_succ.append(i)
        else:
            count_fail +=1
            index_fail.append(i)
        
    return {
        "count_succ": count_succ,
        "queries_succ": queries_succ,
        "l0_dists_succ": l0_dists_succ, 
        "indices_succ": indices_succ,
        "count_fail": count_fail,
        "indices_fails": indices_fails
    }

In [ ]:
evoba_stats = utils.get_evoba_stats(adv_evo_strategy)

In [ ]:
np.mean(evoba_stats["queries_succ"])